In [1]:
import os

In [2]:
%pwd

'/Users/apple/Downloads/Project/Wine-Quality-Prediction-End-to-End-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/apple/Downloads/Project/Wine-Quality-Prediction-End-to-End-Project'

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5  
3      9.8        6  
4      9.4        5

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [8]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [9]:
data.shape

(1599, 12)

In [10]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path               # 📁 যেখানে ডেটা ভ্যালিডেশন সংক্রান্ত সব ফাইল থাকবে
    STATUS_FILE: str            # ✅ স্ট্যাটাস ফাইলের লোকেশন (ডেটা valid কিনা সেটা লেখা থাকবে)
    unzip_data_dir: Path        # 🗃️ আনজিপ করা ডেটাসেটের লোকেশন
    all_schema: dict            # 📊 schema.yaml ফাইল থেকে আনা প্রত্যেকটা কলামের নাম ও টাইপ


In [12]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [13]:
# 🏗️ কনফিগারেশন ম্যানেজমেন্ট ক্লাস
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        # 📄 YAML ফাইলগুলো লোড করা হচ্ছে: config, params, schema
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # 📁 মেইন artifacts ডিরেক্টরি তৈরি করে নিচ্ছে (যদি না থাকে)
        create_directories([self.config.artifacts_root])


    # 🧪 ডেটা ভ্যালিডেশনের কনফিগারেশন রিটার্ন করবে
    def get_data_validation_config(self) -> DataValidationConfig:
        # 🧾 config.yaml এর data_validation সেকশন থেকে config নিচ্ছে
        config = self.config.data_validation
        
        # 📚 schema.yaml এর COLUMNS অংশ থেকে স্কিমা নিচ্ছে
        schema = self.schema.COLUMNS

        # 📂 ডেটা ভ্যালিডেশন সংক্রান্ত ফোল্ডার তৈরি
        create_directories([config.root_dir])

        # ✅ DataValidationConfig অবজেক্ট তৈরি করে তা রিটার্ন করছে
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config


In [14]:
import os
from mlProject import logger

In [15]:
# ✅ ডেটা ভ্যালিডেশন ক্লাস
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        # 🔧 কনফিগারেশন ইনিশিয়ালাইজ
        self.config = config


    def validate_all_columns(self) -> bool:
        try:
            validation_status = None  # 🔁 ভ্যালিডেশন স্ট্যাটাস ট্র্যাক করার জন্য

            # 📥 ডেটা লোড করছে .csv ফাইল থেকে
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)  # 📋 CSV ফাইলের সব কলাম

            # 🧾 schema.yaml থেকে কলামের নাম নিচ্ছে
            all_schema = self.config.all_schema.keys()

            # 🔍 প্রতিটি কলাম যাচাই করছে স্কিমার সাথে মেলে কিনা
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False  # ❌ যদি কোনো কলাম না মেলে
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True  # ✅ কলাম মিলে গেলে
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            # 🔚 ফাইনাল স্ট্যাটাস রিটার্ন
            return validation_status

        except Exception as e:
            # ⚠️ কোনো এরর হলে সেটি রেইজ করে
            raise e


In [16]:
# 🚀 ডেটা ভ্যালিডেশন স্টেজ চালু করার চেষ্টা করছে
try:
    # ⚙️ কনফিগারেশন লোড করছে
    config = ConfigurationManager()
    
    # 📦 ডেটা ভ্যালিডেশনের জন্য কনফিগারেশন নিচ্ছে
    data_validation_config = config.get_data_validation_config()
    
    # 🧪 ডেটা ভ্যালিডেশন অবজেক্ট তৈরি করছে
    data_validation = DataValiadtion(config=data_validation_config)
    
    # ✅ সব কলাম যাচাই করছে স্কিমা অনুযায়ী
    data_validation.validate_all_columns()

# ⚠️ কোনো এরর হলে ধরে ফেলবে এবং রেইজ করবে
except Exception as e:
    raise e


[2025-05-16 11:05:57,237: INFO: common: yaml file: config/config.yaml loaded successfully ✅]
[2025-05-16 11:05:57,246: INFO: common: yaml file: params.yaml loaded successfully ✅]
[2025-05-16 11:05:57,253: INFO: common: yaml file: schema.yaml loaded successfully ✅]
[2025-05-16 11:05:57,259: INFO: common: ✅ ডিরেক্টরি তৈরি হয়েছে: artifacts]
[2025-05-16 11:05:57,282: INFO: common: ✅ ডিরেক্টরি তৈরি হয়েছে: artifacts/data_validation]
